In [1]:
from keras.datasets import mnist,fashion_mnist
(_,_),(X1,y1)=mnist.load_data()
(_,_),(X2,y2)=fashion_mnist.load_data()

데이터 전처리

In [2]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
x_data,tt_x,y_data,tt_y=train_test_split(X1,y1,test_size=0.3,stratify=y1)
tr_x,val_x,tr_y,val_y=train_test_split(x_data,y_data,test_size=0.2,stratify=y_data)
r_tr_x=tr_x.reshape(-1,28, 28,1)
r_tt_x=tt_x.reshape(-1,28, 28,1)
r_val_x=val_x.reshape(-1,28, 28,1)
s_tr_x=r_tr_x/255
s_tt_x=r_tt_x/255
s_val_x=r_val_x/255
s_tr_y=to_categorical(tr_y)
s_tt_y=to_categorical(tt_y)
s_val_y=to_categorical(val_y)
s_tr_x.shape,s_tr_y.shape,s_tt_x.shape,s_tt_y.shape,s_val_x.shape,s_val_y.shape

((5600, 28, 28, 1),
 (5600, 10),
 (3000, 28, 28, 1),
 (3000, 10),
 (1400, 28, 28, 1),
 (1400, 10))

모델 학습

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten
from keras.callbacks import ModelCheckpoint,EarlyStopping
ck_m=ModelCheckpoint('b_cnn_m.h5',monitor='val_acc',save_best_only=True)
es=EarlyStopping(patience=10,restore_best_weights=True)
m=Sequential()
m.add(Conv2D(6,3,padding="same",input_shape=s_tr_x.shape[1:]))
m.add(MaxPooling2D(2))
m.add(Flatten())
m.add(Dense(100,activation='relu'))
m.add(Dense(10,activation='softmax'))
m.compile(optimizer='adam',loss='categorical_crossentropy',metrics='acc')
hy=m.fit(s_tr_x,s_tr_y,validation_data=(s_val_x,s_val_y),epochs=10,callbacks=[es,ck_m])

Epoch 1/10
175/175 [==============================] - 9s 33ms/step - loss: 0.6512 - acc: 0.8148 - val_loss: 0.3318 - val_acc: 0.9000
Epoch 2/10
  1/175 [..............................] - ETA: 7s - loss: 0.1678 - acc: 0.9688

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


175/175 [==============================] - 3s 19ms/step - loss: 0.2803 - acc: 0.9187 - val_loss: 0.2245 - val_acc: 0.9307
Epoch 3/10
175/175 [==============================] - 4s 21ms/step - loss: 0.2048 - acc: 0.9361 - val_loss: 0.1966 - val_acc: 0.9379
Epoch 4/10
175/175 [==============================] - 3s 15ms/step - loss: 0.1638 - acc: 0.9493 - val_loss: 0.1630 - val_acc: 0.9493
Epoch 5/10
175/175 [==============================] - 3s 14ms/step - loss: 0.1207 - acc: 0.9620 - val_loss: 0.1510 - val_acc: 0.9529
Epoch 6/10
175/175 [==============================] - 2s 14ms/step - loss: 0.0918 - acc: 0.9739 - val_loss: 0.1588 - val_acc: 0.9479
Epoch 7/10
175/175 [==============================] - 3s 18ms/step - loss: 0.0692 - acc: 0.9814 - val_loss: 0.1347 - val_acc: 0.9543
Epoch 8/10
175/175 [==============================] - 3s 19ms/step - loss: 0.0535 - acc: 0.9857 - val_loss: 0.1368 - val_acc: 0.9607
Epoch 9/10
175/175 [==============================] - 3s 15ms/step - loss: 0.043

In [5]:
m.evaluate(s_tr_x,s_tr_y),m.evaluate(s_tt_x,s_tt_y),m.evaluate(s_val_x,s_val_y)

44/44 [==============================] - 0s 10ms/step - loss: 0.1244 - acc: 0.9600


([0.023390604183077812, 0.9948214292526245],
 [0.17087599635124207, 0.9543333053588867],
 [0.12441334128379822, 0.9599999785423279])

In [7]:
from keras.models import load_model
l_m=load_model('b_cnn_m.h5')
l_m.evaluate(s_tr_x,s_tr_y),l_m.evaluate(s_tt_x,s_tt_y),l_m.evaluate(s_val_x,s_val_y)

44/44 [==============================] - 0s 6ms/step - loss: 0.1368 - acc: 0.9607


([0.041115716099739075, 0.9889285564422607],
 [0.1756240576505661, 0.9523333311080933],
 [0.13683070242404938, 0.9607142806053162])

예측기 설계

In [ ]:
import numpy as np
def f(m_name,x):
  r_x=x.reshape(-1,28,28,1)
  s_x=r_x/255
  l_m=load_model(m_name)
  out=np.argmax(l_m.predict(s_x),axis=1)
  return out

검증

In [12]:
from sklearn.metrics import classification_report
py=f('b_cnn_m.h5',tt_x)
ty=tt_y
print(classification_report(ty,py))

94/94 [==============================] - 1s 10ms/step
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       294
           1       0.99      0.99      0.99       340
           2       0.95      0.95      0.95       310
           3       0.97      0.90      0.93       303
           4       0.95      0.94      0.94       295
           5       0.96      0.96      0.96       268
           6       0.96      0.97      0.96       287
           7       0.96      0.94      0.95       308
           8       0.94      0.95      0.95       292
           9       0.88      0.96      0.92       303

    accuracy                           0.95      3000
   macro avg       0.95      0.95      0.95      3000
weighted avg       0.95      0.95      0.95      3000

